# 01_scrap

- scrapping data from thestandard.co and upload to DataPlatform

## Getting data from thestandard.co

In [ ]:
import pandas as pd
import bs4
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
title_list = []
date_list = []
detail_list = []

In [ ]:
page = 1
while page <= 5:
  data = requests.get('https://thestandard.co/page/'+str(page)+'?s=%E0%B8%AD%E0%B8%B8%E0%B8%9A%E0%B8%B1%E0%B8%95%E0%B8%B4%E0%B9%80%E0%B8%AB%E0%B8%95%E0%B8%B8&search=')
  soup = bs4.BeautifulSoup(data.text,"html.parser")
  for f in soup.find_all('div', class_='col-md-8'):
    title_list.append(f.find('h3', class_= 'news-title').find('a').text.strip())
    date_list.append(f.find('div', class_='date').text.strip())
    detail_list.append(f.find('div', class_='desc').text.strip())
  page += 1

In [ ]:
table = pd.DataFrame([title_list,date_list,detail_list]).transpose()

In [ ]:
table.columns = ['TITLE', 'date', 'DETAIL']

In [ ]:
table.set_index('TITLE')

In [ ]:
table

In [ ]:
table['date_split'] = table['date'].str.split('\s')

In [ ]:
table['D'] = table['date_split'].str[0]
table['M'] = table['date_split'].str[1]
table['Y']  = table['date_split'].str[2]

In [ ]:
table['M'].unique()

In [ ]:
table['M'].replace({
    'มกราคม':'01',
    'กุมภาพันธ์':'02',
    'มีนาคม' :'03',
    'เมษายน':'04',
    'พฤษภาคม':'05',
     'มิถุนายน':'06',
     'กรกฎาคม':'07',
    'สิงหาคม':'08',
    'กันยายน':'09',
    'ตุลาคม':'10',
    'พฤศจิกายน':'11',
    'ธันวาคม': '12'
    }, inplace=True)

In [ ]:
table['DATE'] = table['D']+'-'+table['M']+'-'+table['Y']

In [ ]:
table['DATE'] = pd.to_datetime(table['DATE'])

In [ ]:
table.drop(['date', 'date_split', 'D','M','Y'], axis=1, inplace=True)

In [ ]:
table.info()

In [ ]:
table

In [ ]:
table.to_csv(r'week04.csv',index=False,encoding='utf-8')

## Upload data to DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_raw_id = 64

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='week04.csv')